<a href="https://colab.research.google.com/github/ak5605/Language-Models/blob/main/Bi_LSTM_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


In [8]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

# Load the dataset from HuggingFace
dataset = load_dataset("carblacac/twitter-sentiment-analysis")

# Print the features of the dataset to identify column names
print(dataset['train'].features)

# Assuming you have loaded the dataset into 'dataset'
train_data = dataset['train']

# Identify the correct column names and extract features and labels
feature_column_name = 'text'
target_column_name = 'feeling'

train_texts = train_data[feature_column_name]
train_labels = train_data[target_column_name]

# Convert labels to NumPy array
train_labels = np.array(train_labels)

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Tokenize and pad the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=100, padding='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(test_texts), maxlen=100, padding='post')

# Build a Bi-LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Bi-LSTM model
model.fit(X_train_seq, train_labels, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
accuracy_lstm = model.evaluate(X_test_seq, test_labels)[1]

print(f'Bi-LSTM Accuracy: {accuracy_lstm}')


{'text': Value(dtype='string', id=None), 'feeling': Value(dtype='int32', id=None)}
Epoch 1/5
2700/2700 [==============================] - 871s 321ms/step - loss: 0.4840 - accuracy: 0.7672 - val_loss: 0.4532 - val_accuracy: 0.7844
Epoch 2/5
2700/2700 [==============================] - 866s 321ms/step - loss: 0.2964 - accuracy: 0.8750 - val_loss: 0.4889 - val_accuracy: 0.7684
Epoch 3/5
2700/2700 [==============================] - 870s 322ms/step - loss: 0.1722 - accuracy: 0.9313 - val_loss: 0.5711 - val_accuracy: 0.7646
Epoch 4/5
2700/2700 [==============================] - 868s 321ms/step - loss: 0.1090 - accuracy: 0.9564 - val_loss: 0.6732 - val_accuracy: 0.7581
Epoch 5/5
750/750 [==============================] - 22s 29ms/step - loss: 0.8122 - accuracy: 0.7525
Bi-LSTM Accuracy: 0.7524793744087219


In [9]:


# Define a function for preprocessing the custom input
def preprocess_input(text, tokenizer):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')
    return padded_sequence

# Example custom input
custom_input = "nlp text book is good"

# Preprocess the custom input
custom_input_seq = preprocess_input(custom_input, tokenizer)

# Make predictions using the trained model
prediction = model.predict(custom_input_seq)

# Interpret the prediction
sentiment_label = "Positive" if prediction > 0.5 else "Negative"

# Print the result
print(f"Predicted Sentiment: {sentiment_label} (Confidence: {prediction[0][0]:.2f})")


1/1 [==============================] - 1s 1s/step
Predicted Sentiment: Positive (Confidence: 1.00)
